# Music Practice Builder
- Load data
- create empty practice session
- split into essential and non-essential items
- add essential items to session
- sort by priority


## Configuration

In [ ]:
import math
import random
import numpy as np
import pandas as pd

%matplotlib inline

In [ ]:
#input_file = './practice_elements.xlsx'
input_file = '~/Dropbox/practice_elements.xlsx'
practice_time_minutes = 45
category_item_limits_time_block_minutes = 30  # Interpret the category item limits as being per a time block

## Load the data

In [ ]:
categories = pd.read_excel(
    input_file,
    sheetname='categories',
    index_col=0,
    converters=
    {
        'min_items': int,
        'max_items': int,
    })

In [ ]:
data = pd.read_excel(
    input_file, 
    sheetname='items',
    converters=
    {
        'min_time': int,
        'max_time': int,
        'priority': float,
        'essential': bool,
        'tempo': str,
        'notes': str,
    })

Fill in missing values with sensible defaults:

In [ ]:
data.weight = data.weight.fillna(1)
data.min_time = data.min_time.fillna(2)
data.max_time = data.max_time.fillna(5)
data.sort_order = data.sort_order.fillna(2)
data.tempo = data.tempo.fillna('')
data.notes = data.notes.fillna('')

The max_items category values are per a minimal practice time. Adjust them now:

In [ ]:
category_item_limit_scale = max(1, round(practice_time_minutes / category_item_limits_time_block_minutes))
categories.min_items *= category_item_limit_scale
categories.max_items *= category_item_limit_scale
categories

## Generate the random item times

In [ ]:
def generate_random_times(df):
    return pd.DataFrame(
        {'time': df.apply(lambda row: random.randrange(row.min_time, row.max_time+1), axis=1)}, 
        index=df.index)

In [ ]:
data = data.join(generate_random_times(data))

## Populate the session with the essential items

In [ ]:
session = data.query('essential == True')

## Extract the set of candidate items

In [ ]:
items = data.query('essential == False and weight > 0')

## Process the category minimum item count constraint

In [ ]:
for category, group in items.groupby('category'):
    # For this category, attempt to select the min number of items
    try:
        min_items = categories.loc[category].min_items
        current = len(session[session.category == category])
        required = min(min_items - current, len(group))
        if not np.isnan(required) and required > 0:
            new_items = group.sample(n=required, weights='weight')
            items = items.drop(new_items.index)
            session = session.append(new_items)
    except:
        pass

## Fill the rest of the session

In [ ]:
while session.time.sum() < practice_time_minutes and len(items) > 0:
    # Clean out any maxed categories from the candidate items
    for category, group in items.groupby('category'):
        current_items_in_category = len(session[session.category == category])
        max_category = categories.loc[category].max_items
        if not np.isnan(max_category) and current_items_in_category >= max_category:
            print('Category "{0}" reached maximum item count ({1})'.format(category, max_category))
            items = items[items.category != category]
        
    # only query candidates where the min time can fit into the remaining time
    remaining_time = practice_time_minutes - session.time.sum()
    candidates = items.query('min_time <= {0}'.format(remaining_time))
    
    # If no candidates left, we must abort
    if len(candidates) == 0:
        print('unable to fill session')
        break
        
    # pick the next item
    i = candidates.sample(n=1, weights='weight')
    
    item_time = i.time.iloc[0]
    item_min_time = i.min_time.iloc[0]
    item_max_time = i.max_time.iloc[0]
        
    if item_time <= remaining_time:
        # if the item fits, use it
        session = session.append(i)
        items = items.drop(i.index)
    else:
        # trim the item time to the remaining (without exceeding item cap)
        print('setting time to remaining')
        i.loc[:,'time'] = min(remaining_time, item_max_time)
        session = session.append(i)
        items = items.drop(i.index)

# Today's Practice Session

In [ ]:
print('Target time: {0}, Actual time: {1}'.format(practice_time_minutes, session.time.sum()))

In [ ]:
session.sort_values(by='sort_order')[['name', 'category', 'tempo', 'notes', 'time']]

In [ ]:
session.groupby('category').name.count().plot()

In [ ]:
session.groupby('category').time.sum().plot()